In [1]:
import pandas as pd
import requests


In [4]:
# This is the URL where NOAA serves the current prediction stations data in XML format.
stations_url = "https://opendap.co-ops.nos.noaa.gov/axis/webservices/currentpredictionstations/response.jsp?format=xml"


In [5]:
response = requests.get(stations_url)


In [6]:
print("Status code:", response.status_code)


Status code: 200


In [7]:
xml_content = response.text[:1000]  # first 1000 characters
print(xml_content)


<?xml version="1.0" encoding="utf-8"?><soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><soapenv:Body><CurrentPredictionStations xmlns="https://opendap.co-ops.nos.noaa.gov/axis/webservices/currentpredictionstations/wsdl"><stations><station ID="ACT0091" name="Eastport, Friar Roads"><stationBin><binNumber>1</binNumber><depth></depth></stationBin><metadata><location><lat>44.9000</lat><long>-66.9833</long></location><stationType>Subordinate</stationType></metadata></station><station ID="ACT0101" name="Western Passage, off Kendall Head"><stationBin><binNumber>1</binNumber><depth></depth></stationBin><metadata><location><lat>44.9317</lat><long>-67.0000</long></location><stationType>Subordinate</stationType></metadata></station><station ID="ACT0106" name="Western Passage, off Frost Ledge"><stationBin><binNumber>1</binNumber><depth></depth></stationBin><metadata><locatio

In [9]:
namespaces = {
    "soapenv": "http://schemas.xmlsoap.org/soap/envelope/",
    "ns": "https://opendap.co-ops.nos.noaa.gov/axis/webservices/currentpredictionstations/wsdl"
}

stations_df = pd.read_xml(
    response.content,
    xpath=".//ns:station",
    namespaces=namespaces
)

stations_df.head()


,ID,name,stationBin,metadata
0,ACT0091,"Eastport, Friar Roads",NaN,NaN
1,ACT0101,"Western Passage, off Kendall Head",NaN,NaN
2,ACT0106,"Western Passage, off Frost Ledge",NaN,NaN
3,ACT0111,"Pond Point, 7.6 miles SSE of",NaN,NaN
4,ACT0116,"Moosabec Reach, east end",NaN,NaN


In [10]:
import xml.etree.ElementTree as ET
from pandas import json_normalize

# Parse the XML content
root = ET.fromstring(response.content)

# Define namespaces again
namespaces = {
    "soapenv": "http://schemas.xmlsoap.org/soap/envelope/",
    "ns": "https://opendap.co-ops.nos.noaa.gov/axis/webservices/currentpredictionstations/wsdl"
}

stations_list = []

for station in root.findall(".//ns:station", namespaces):
    station_dict = {
        "station_id": station.attrib.get("ID"),
        "station_name": station.attrib.get("name"),
        "bin_number": station.findtext(".//ns:binNumber", namespaces=namespaces),
        "depth": station.findtext(".//ns:depth", namespaces=namespaces),
        "latitude": station.findtext(".//ns:lat", namespaces=namespaces),
        "longitude": station.findtext(".//ns:long", namespaces=namespaces),
        "station_type": station.findtext(".//ns:stationType", namespaces=namespaces)
    }
    stations_list.append(station_dict)

# Create a DataFrame
stations_df = pd.DataFrame(stations_list)
stations_df.head()


,station_id,station_name,bin_number,depth,latitude,longitude,station_type
0,ACT0091,"Eastport, Friar Roads",1,,44.9000,-66.9833,Subordinate
1,ACT0101,"Western Passage, off Kendall Head",1,,44.9317,-67.0000,Subordinate
2,ACT0106,"Western Passage, off Frost Ledge",1,,44.9650,-67.0317,Subordinate
3,ACT0111,"Pond Point, 7.6 miles SSE of",1,,44.3350,-67.5033,Subordinate
4,ACT0116,"Moosabec Reach, east end",1,,44.5285,-67.5727,Subordinate
